In [1]:
%matplotlib qt

In [2]:
import os, pyxdf, json
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mne.time_frequency import psd_welch
from mne.decoding import Scaler, Vectorizer

from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import RepeatedStratifiedKFold, HalvingGridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

### Settings

In [3]:
# Files
subject = 'P00J'
session = 1
# task = 'MI-hands'
task = 'P300_4x4_gface'
lslDir = os.path.join(os.path.expanduser('~'), 'Documents\CurrentStudy')

# LSL Stream
eeg_stream_type = 'EXG'
markers_stream_type = 'Marker'

# Events
if task == 'MI-hands':
    event_dict = {'rest': 0, 'MI/hands': 1}
    rest_duration = 5
    task_duration = 5
    tmin = 0
    tmax = 5
elif 'P300' in task:
    event_dict = {'nontarget': 0, 'target': 1}
    task_duration = 1
    tmin = 0
    tmax = 1

# Plot
plotGraphs = True
scalings = dict(eeg=200e-6)
plot_duration = 10

# Bandpass filter
bp_l_freq = 0.1
bp_h_freq = 40.

# Features
features = 'time'
# features = 'psd'

In [4]:
# Create sub-folder for data
fif_dir = os.path.join('fif')
if not os.path.exists(fif_dir):
    os.makedirs(fif_dir)

### Find LSL Files

In [5]:
# Find files
xdf_files = []
hasSubject = subject!=''
hasSession = session!=''
hasTask = task!=''
for root, dir, files in os.walk(lslDir):
    for file in files:
        validFile = True
        if hasSubject:
            validFile = validFile and (('sub-'+subject) in file)
        if hasSession:
            validFile = validFile and (('ses-S' + str(session).zfill(3)) in file)
        if hasTask:
            validFile = validFile and (('task-' + task) in file)
        validFile = validFile and file.endswith('.xdf')
        if validFile:
            print(file)
            matchingFile = os.path.join(root, file)
            xdf_files.append(matchingFile)

if len(xdf_files) == 0:
    print('No files found')

sub-P00J_ses-S001_task-P300_4x4_gface_run-001_eeg.xdf
sub-P00J_ses-S001_task-P300_4x4_gface_run-002_eeg.xdf
sub-P00J_ses-S001_task-P300_4x4_gface_run-003_eeg.xdf
sub-P00J_ses-S001_task-P300_4x4_gface_run-004_eeg.xdf
sub-P00J_ses-S001_task-P300_4x4_gface_run-005_eeg.xdf


In [6]:
# Parse streams
eeg_stream, marker_stream = [], []

print('Parsing streams')
for xdf_file in xdf_files:
    streams, header = pyxdf.load_xdf(xdf_file)
    for i in range(len(streams)):
        if streams[i]['info']['type'][0] == eeg_stream_type:
            print("Found %s stream in %s" % (eeg_stream_type, os.path.basename(xdf_file)))
            eeg_stream.append(streams[i])
        elif streams[i]['info']['type'][0] == markers_stream_type:
            print("Found %s stream in %s" % (markers_stream_type, os.path.basename(xdf_file)))
            marker_stream.append(streams[i])
del streams, header

Parsing streams
Found EXG stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-001_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-001_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-002_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-002_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-003_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-003_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-004_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-004_eeg.xdf
Found EXG stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-005_eeg.xdf
Found Marker stream in sub-P00J_ses-S001_task-P300_4x4_gface_run-005_eeg.xdf


### Extract EEG and Marker data

In [7]:
# Extract EEG Info
print("Extracting EEG info")

ch_names = []
if eeg_stream[0]['info']['desc'][0]:
    print("EEG channel names found")
    for i in range(len(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'])):
        ch_names.append(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0])
else:
    print("EEG channel names not found... setting default")
    ch_names = ['FP1', 'FP2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']
print('Channels: ', ch_names)

sfreq = float(eeg_stream[0]['info']['nominal_srate'][0])
print('Sampling frequency: ', sfreq)

# Create MNE info object
eeg_info = mne.create_info(ch_names, sfreq, ch_types='eeg')

Extracting EEG info
EEG channel names not found... setting default
Channels:  ['FP1', 'FP2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']
Sampling frequency:  125.0


In [8]:
# Setup Montage
montage = mne.channels.read_custom_montage('openbci_montage.elc')
# montage.plot()

In [9]:
# Get all EEG data
eeg_raw_list = []

for n in range(len(eeg_stream)):
    # Create MNE Raw object
    eeg_data = np.transpose(eeg_stream[n]['time_series'])
    eeg_data = eeg_data / 1e6
    print(eeg_data.shape)
    eeg_raw = mne.io.RawArray(eeg_data, eeg_info)
    
#     # Set FP1 and FP2 as EOG channels
#     eeg_raw = eeg_raw.set_channel_types({'FP1': 'eog', 'FP2': 'eog'})
#     eeg_raw.pick_types(eeg=True, eog=True)
    
    # Set montage
    eeg_raw = eeg_raw.set_montage(montage)

    # Add annotations
    onset, duration, description = [], [], []
    current_target = -1
    current_flash = -1
    for i in range(len(marker_stream[n]['time_series'])):
        if 'MI' in task:
            if ('rest' in marker_stream[n]['time_series'][i][0]):
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(rest_duration)
                description.append(marker_stream[n]['time_series'][i][0])
            elif ('task' in marker_stream[n]['time_series'][i][0]):
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(task_duration)
                description.append(marker_stream[n]['time_series'][i][0].replace('task_', '').replace('-','/'))
        elif 'P300' in task:
            if('target' in marker_stream[n]['time_series'][i][0]):
                current_target = json.loads(marker_stream[n]['time_series'][i][0])['target']
            elif('flash' in marker_stream[n]['time_series'][i][0]):
                current_flash = json.loads(marker_stream[n]['time_series'][i][0])['flash']
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(task_duration)
                description.append("target" if current_flash == current_target else "nontarget")
    annotations = mne.Annotations(onset, duration, description)
    eeg_raw = eeg_raw.set_annotations(annotations)
    
    # Create list of raw objects
    eeg_raw_list.append(eeg_raw)

(16, 21304)
Creating RawArray with float64 data, n_channels=16, n_times=21304
    Range : 0 ... 21303 =      0.000 ...   170.424 secs
Ready.
(16, 21411)
Creating RawArray with float64 data, n_channels=16, n_times=21411
    Range : 0 ... 21410 =      0.000 ...   171.280 secs
Ready.
(16, 21048)
Creating RawArray with float64 data, n_channels=16, n_times=21048
    Range : 0 ... 21047 =      0.000 ...   168.376 secs
Ready.
(16, 20686)
Creating RawArray with float64 data, n_channels=16, n_times=20686
    Range : 0 ... 20685 =      0.000 ...   165.480 secs
Ready.
(16, 20309)
Creating RawArray with float64 data, n_channels=16, n_times=20309
    Range : 0 ... 20308 =      0.000 ...   162.464 secs
Ready.


In [10]:
# Concatenate raw objects
raw = mne.concatenate_raws(eeg_raw_list)
raw

<RawArray | 16 x 104758 (838.1 s), ~12.8 MB, data loaded>

### Pre-processing

In [11]:
# Common average reference
raw_orig = raw.copy()
raw = raw.set_eeg_reference('average', projection=False)

if plotGraphs:
    fig = raw_orig.plot(title='Before Re-referencing', n_channels=16, scalings=scalings)
    fig = raw.plot(title='After Re-referencing', n_channels=16, scalings=scalings)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Using matplotlib as 2D backend.


In [12]:
# Bandpass filter data
raw_orig = raw.copy()
raw = raw.filter(l_freq=bp_l_freq, h_freq=bp_h_freq)

if plotGraphs:
    fig = raw_orig.plot(title='Before Filtering', scalings=scalings, duration=plot_duration)
    fig = raw.plot(title='After Filtering', scalings=scalings, duration=plot_duration)

Filtering raw data in 5 contiguous segments
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 4125 samples (33.000 sec)



### ICA Artifact Removal

In [13]:
raw_orig = raw.copy()

# filter data to remove slow drifts
raw_filt = raw.copy()
raw_filt.filter(l_freq=1., h_freq=None)

# ICA decomposition
ica = mne.preprocessing.ICA(n_components=16, method='fastica', max_iter=200, random_state=42, verbose=True)
ica = ica.fit(raw_filt)

Filtering raw data in 5 contiguous segments
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 413 samples (3.304 sec)

Fitting ICA to data using 16 channels (please be patient, this may take a while)
Selecting by number: 16 components
Fitting ICA took 2.3s.


C:\Users\Jason\AppData\Local\Temp\ipykernel_4636\895341945.py:9: RuntimeWarning: Using n_components=16 (resulting in n_components_=16) may lead to an unstable mixing matrix estimation because the ratio between the largest (11) and smallest (5.7e-26) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 15
  ica = ica.fit(raw_filt)


In [14]:
# Plot ICA sources
fig = ica.plot_sources(raw_orig)

Creating RawArray with float64 data, n_channels=16, n_times=104758
    Range : 0 ... 104757 =      0.000 ...   838.056 secs
Ready.


In [15]:
# Select source that corresponds to artifact and remove it
# ica.exclude = [0]
# ica.exclude = [2]
print('ICA sources to exclude: ', ica.exclude)

ICA sources to exclude:  []


In [16]:
ica.apply(raw)

if plotGraphs:
    fig = raw_orig.plot(title='Before ICA', scalings=scalings, duration=plot_duration)
    fig = raw.plot(title='After ICA', scalings=scalings, duration=plot_duration)

Applying ICA to Raw instance
    Transforming to ICA space (16 components)
    Zeroing out 0 ICA components
    Projecting back using 16 PCA components


### Epoch data

In [17]:
# Epoch data
events, event_id = mne.events_from_annotations(raw, event_id=event_dict)
epoch = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, picks='eeg', preload=True)
print(epoch)

Used Annotations descriptions: ['nontarget', 'target']
Not setting metadata
Not setting metadata
4000 matching events found
No baseline correction applied
0 projection items activated
Loading data for 4000 events and 126 original time points ...
0 bad epochs dropped
<Epochs |  4000 events (all good), 0 - 1 sec, baseline off, ~61.6 MB, data loaded,
 'nontarget': 3750
 'target': 250>


### Features

In [22]:
evoked_target = epoch['target'].average()
fig = evoked_target.plot_joint(times=[0., 0.3, 0.4, 0.6, 0.7], picks=['C3', 'C4', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2'])

No projector specified for this dataset. Please consider the method self.add_proj.


In [ ]:
# Labels
y = epoch.events[:,-1] - min(epoch.events[:,-1])

# Time-Domain Features
if features == 'time':
    scaler = Scaler(epoch.info)
    X = scaler.fit_transform(epoch.get_data())
    
#     if ('P300' in task) and plotGraphs:
#         evoked_target = eeg_epoch['target'].average()
#         evoked_target.plot()
    
# Frequency-Domain Features
elif features == 'psd':
    psds, freqs = psd_welch(epoch, average='mean', fmin=bp_l_freq, fmax=bp_h_freq, n_jobs=-1)
    X = 10 * np.log10(psds)
#     X = psds / np.sum(psds, axis=-1, keepdims=True)
    
    if ('MI' in task) and plotGraphs:
        sel_chs = [2, 3, 4, 5, 6, 7, 14, 15]
        psd_means_class_0 = np.transpose(np.mean(X[y==0], axis=0))
        psd_means_class_1 = np.transpose(np.mean(X[y==1], axis=0))
        psd_means_class_0 = psd_means_class_0[:,sel_chs]
        psd_means_class_1 = psd_means_class_1[:,sel_chs]
        
        fig = plt.figure()
        ax = fig.add_subplot(111)
        for i in range(len(sel_chs)):
            line = ax.plot(freqs, psd_means_class_0[:,i], ':', label=ch_names[sel_chs[i]] + ' Rest')
            ax.plot(freqs, psd_means_class_1[:,i], '-', label=ch_names[sel_chs[i]] + ' MI-hands', color=line[0].get_color())
        ax.set(title='Welch PSD', xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
        ax.set_ylim(bottom=-135, top=-85)
        ax.legend(loc='best')

# Vectorize features
if len(X.shape) > 2:
    print('Vectorizing features to 2D...')
    print('Original X.shape: ', X.shape)
    vec = Vectorizer()
    X = vec.fit_transform(X)

print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

### Classification

In [ ]:
# Set up cross validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up scoring
scoring = 'accuracy'
scores = {'Classifier': [],
          'Score': [],
          'Std': []
         }

In [ ]:
# Set up Classifiers
classifiers = []

# KNN
params = {}
params['n_neighbors'] = np.arange(2,11,1)
params['weights'] = ['uniform', 'distance']
clf = HalvingGridSearchCV(KNeighborsClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['KNN', clf, params])

# DT
params = {}
params['criterion'] = ['gini', 'entropy']
params['min_samples_split'] = np.arange(2,11,2)
clf = HalvingGridSearchCV(DecisionTreeClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['DT', clf, params])

# RF
params = {}
params['criterion'] = ['gini', 'entropy']
params['n_estimators'] = (10, 20, 30)
params['min_samples_split'] = np.arange(2,11,2)
clf = HalvingGridSearchCV(RandomForestClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['RF', clf, params])

# LDA
params = {}
params['solver'] = ['svd']
clf = HalvingGridSearchCV(LinearDiscriminantAnalysis(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['LDA', clf, params])

# SVM
params = {}
params['C'] = (1e-4, 1e-2, 1)
params['gamma'] = (1e-4, 1e-2, 1, 10)
params['kernel'] = ['linear', 'rbf']
clf = HalvingGridSearchCV(SVC(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['SVM', clf, params])

# SGD
params = {}
params['loss'] = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
params['penalty'] = ['l2', 'l1', 'elasticnet']
params['alpha'] = (1e-4, 1e-2, 1, 10)
clf = HalvingGridSearchCV(SGDClassifier(), param_grid=params, n_jobs=-1, cv=cv, scoring=scoring)
classifiers.append(['SGD', clf, params])

In [ ]:
# Train Classifiers
for c in range(len(classifiers)):
    clf_name = classifiers[c][0]
    clf = classifiers[c][1].fit(X, y)
    print("Training %s..." % clf_name)
    print('%s score: %2.2f' % (clf_name, clf.best_score_))
    print('%s std  : %2.2f' % (clf_name, np.mean(clf.cv_results_['std_test_score'])))
    print()
    scores['Classifier'].append(clf_name)
    scores['Score'].append(clf.best_score_)
    scores['Std'].append(np.mean(clf.cv_results_['std_test_score']))

In [ ]:
# Score summary
df = pd.DataFrame(scores)
df

In [ ]:
# Best Classifier
print('Best Classifier:')
df.loc[df['Score'].idxmax()]